<a href="https://colab.research.google.com/github/Ziqi-Li/GIS5122/blob/main/assignments/GIS5122_A2_sp25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2
**Due date/time:** Feb 14th Fri 23:59pm

**Instruction**: Make a copy of this notebook and read through the questions and instructions. Write down code/text to perform the analysis.

**Submission**: After you finish working on this notebook, click File->Download->Download .ipynb

Please submit a notebook named A2_your_name.ipynb on Canvas.

**Environment**: Feel free to use either your local python environment or Google Colab.

## Detailed instructions:

In this assignment, we will practie on model selection and spatial autocorrelation.

**Note**: From this assignment and onward, if you are interested in performing your analysis using your own data (e.g., you may have a dataset from a research project), you are very encouraged to do so. But it would be great if you let me know in advance.

We will be using a county-level health data `health` obtained from [CDC](https://www.cdc.gov/places/index.html) to investigate [Social Determinants of Health](https://www.cdc.gov/about/sdoh/index.html). You can find the explanations to the columns in the data [here](https://raw.githubusercontent.com/Ziqi-Li/GIS5122/main/data/cdc_county_health.txt).

You are also given a county-level socio-demographic dataset `socio_demo` obtaiend from the American Community Survey (2015-2019) dataset. Variable name explanations can be seen [here](https://github.com/Ziqi-Li/GIS5122/blob/main/data/acs_2019_variables.txt).

The objective is to identify socio-demographic factors that will explain health outcome inequality in the US, by estimating a linear regression model and examine its assumptions and interpretations.

You are also given a county-level boundary file `counties` for making necessary maps.

Below are specific tasks you need to perform and their credits (total 100%):
1. Create a `GeoDataFrame` by merging the `health`, the `socio_demo` and the `counties` boundary together. Code is given.
2. Choose **ONE** health outcome of your interest. Make a map of the health outcome to show its spatial distribution. (**5%**)
3. Build a linear regression model (OLS) with the health outcome as the dependent variable (y), and some socio-demographic variables as the predictor variables (X). You are supposed to tweak the model specification based on your own understanding of the potential relationships.
4. Calculate VIF values to make sure no obvious multicolinearity presents (drop predictors with VIF > 10 sequentially). (**15%**)
5. Create a residual vs. fit plot and a Q-Q plot to check model residual assumptions. Describe whether the patterns in these plots noticeably violate any of the LINE regression assumptions. (**20%**)
6. Report and interpret your linear regression results regarding model R2, regression coefficients, and statistical significance of the coefficients. (**40%**)
7. Make a map of the residuals to examine spatial autocorrelation. Then compute Moran's I value of the residuals based on Queen-contiguity spatial weights matrix. Interpret the Moran's I value and its statistical significance. (**20%**)



You may need to install these two packages for Moran's I.

In [1]:
pip install -q libpysal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 20.2 MB/s eta 0:00:00


In [2]:
pip install -q splot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.4/215.4 kB 6.8 MB/s eta 0:00:00


Import all the packages needed to run the analysis

In [3]:
import geopandas as gpd
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm

from libpysal.weights import Queen
from splot.esda import plot_moran
from esda.moran import Moran
import spreg

Code to read in data and merge them into one final `GeoDataFrame`.

In [4]:
health_url = "https://raw.github.com/Ziqi-Li/GIS5122/main/data/cdc_health_county.csv"
health = pd.read_csv(health_url)

In [5]:
socio_demo_url = "https://raw.github.com/Ziqi-Li/GIS5122/main/data/acs_2019.csv"
socio_demo = pd.read_csv(socio_demo_url)

In [6]:
counties_url = "https://raw.github.com/Ziqi-Li/gis5122/master/data/us_counties.geojson"
counties = gpd.read_file(counties_url)

In [7]:
income_health = pd.merge(socio_demo, health, left_on = "county_id", right_on = "LocationID")

In [8]:
gdf_income_health = pd.merge(counties, income_health, on = "county_id")

Your code will start here, and your analysis will be based on this GeoDataFrame `gdf_income_health`.